In [ ]:
import pandas as pd
import json

df = pd.read_csv('s3://your-s3-bucket/bls-data/pr.data.0.Current', delim_whitespace=True)
df['value'] = pd.to_numeric(df['value'], errors='coerce')

with open('us-population.json') as f:
    raw = json.load(f)

headers, *rows = raw
pop_df = pd.DataFrame(rows, columns=headers)
pop_df['POP'] = pop_df['POP'].astype(int)
pop_df['time'] = pop_df['time'].astype(int)

subset_pop = pop_df[pop_df['time'].between(2013, 2018)]
print("Mean:", subset_pop['POP'].mean())
print("Std Dev:", subset_pop['POP'].std())

df = df[df['period'].str.startswith('Q')]
df_grouped = df.groupby(['series_id', 'year'])['value'].sum().reset_index()
best_years = df_grouped.loc[df_grouped.groupby('series_id')['value'].idxmax()]
print(best_years)

filtered = df[(df['series_id'] == 'PRS30006032') & (df['period'] == 'Q01')]
filtered['year'] = filtered['year'].astype(int)
joined = filtered.merge(pop_df, left_on='year', right_on='time', how='left')
joined = joined[['series_id', 'year', 'period', 'value', 'POP']].rename(columns={'POP': 'Population'})
print(joined)